# Слайсы

**Слайс** - это структура данных переменной длины, элементы которой имеют одинаковый тип. Тип слайс выглядит как **[]T**, где T является типом его элементов.

Массивы и слайсы очень тесно связаны между собой. Слайс даёт доступ к подпоследовательности(возможно всей) элементов массива, который называют **базовым массивом**(underlying array) слайса. Слайс состоит из трёх компонент:
 - указателя
 - длины
 - вместительности(capacity)

**Указатель** указывает на первый элемент массива, который достижим через этот слайс и он необъязательно должен быть первым элементом массива. **Длина** - это количество элемент в слайсе. Она не может быть больше вместительности. **Вместительность** - это обычно количество элементов между началом слайса и концом базового массива. Встроенные функции **len** и **cap** позволяют получить длину и вместительность слайса.

Несколько слайсов могуть указывать на один и тот же базовый массив и даже на перекрывающиеся его участки.

**Слайс-оператор**(slice operator) s[i:j], где 0 <= i <= j <= cap(s), создаёт новый слайс, который указывает на элементы с i по j-1 последовательности s, где s может быть массивом, указателем на массив или другим слайсом. Полученный слайс состоит из j-i элементов. Если не указан i, то вместо него используется 0, а если не указан j - len(s). Пример:

In [1]:
import "fmt"

// Массив.
var days [7]string = [7]string{
    "Понедельник",
    "Вторник",
    "Среда",
    "Черверг",
    "Пятница",
    "Суббота",
    "Воскресенье",
}

// Слайсы.
weekdays := days[:5]
vacation := days[4:]

fmt.Printf("%T, len: %d, cap: %d, %v\n", weekdays, len(weekdays), cap(weekdays), weekdays)
fmt.Sprintf("%T, len: %d, cap: %d, %v\n", vacation, len(vacation), cap(vacation), vacation)

[]string, len: 5, cap: 7, [Понедельник Вторник Среда Черверг Пятница]
[]string, len: 3, cap: 3, [Пятница Суббота Воскресенье]



Попытка получить слайс, который выходит за вместительность приведёт к ошибке во время исполнения(runtime error), но можно получить слайс, который выходит за длину:

In [2]:
fmt.Sprint(weekdays[:6])

[Понедельник Вторник Среда Черверг Пятница Суббота]


При передаче слайса функции создаётся копия слайса, **но не базового массива**, на который указывает этот слайс. Поэтому изменения, внесённые внутри функции видны и за её пределами:

In [3]:
func setAll(s []int, value int) {
    for i := range s {
        s[i] = value
    }
}

nums := []int{1, 2, 3}
setAll(nums, 7)
fmt.Sprint(nums)

// Создаём массив.
array := [...]int{0, 1, 2, 3}
// Операция получения слайса применима и к массиву. В этом случае создаётся
// слайс, базовым массивом которого является указанный слайс.
setAll(array[:], 9)
fmt.Sprint(array)

[9 9 9 9]


**Литерал слайса** похож на литерал массива, но для него не указывается длина. Как и в случае с литералами массивов, литерал слайса позволяет создавать элементы слайса по порядку, с указанием значения каждого отдельного элемента или использованием одновременно обоих этих подходов:

In [4]:
s := []string{"green", "red", "brown", "yellow"}
fmt.Sprint(len(s), " ", cap(s), "\n", s)

4 4
[green red brown yellow]


В отличие от массивов слайсы **нельзя сравнивать** между собой. В стандартную библиотеку входит функция bytes.Equal для сравнения двух слайсов байт([]byte), но в остальных случаях мы должно создавать свои функции.

Слайс можно сравнивать **только** с nil. Нулевым значением для слайса является nil. nil слайс не имеет базового массива, его длина и вместительность равны 0. Из равенства длины и вместительности слайса 0 **не следует**, что это nil слайс. Примерами таких слайсов являются []int{} или make([]int, 3)[3:]:

In [5]:
var s []int
fmt.Printf("len: %d, cap: %d, s == nil: %t\n", len(s), cap(s), s == nil)

s = []int{}
fmt.Sprintf("len: %d, cap: %d, s == nil: %t", len(s), cap(s), s == nil)

len: 0, cap: 0, s == nil: true
len: 0, cap: 0, s == nil: false


Поэтому для проверки слайса на пустоту необходимо использовать функцию len, а не s == nil.

Встроенная функция **make** позволяет создать слайс заданной длины и вместительности. Вместительность можно не указывать. В этом случае она устанавливается равной длине:
 - make([]T, len) - создание слайса длины и вместительности len.  
 - make([]T, len, cap) - создание слайса длины len и вместительности cap.
 
За кулисами функция make создаёт массив и возвращает его слайс.

## Функция append

Встроенная функция append позволяет добавить элемент к слайсу:

In [6]:
var runes []rune
for _, r := range "Hello, Мир" {
    runes = append(runes, r)
}
fmt.Sprintf("%q", runes)

['H' 'e' 'l' 'l' 'o' ',' ' ' 'М' 'и' 'р']


Здесь мы получаем руны, из которых состоит строка. Этот пример используется только для демонстрации, более коротко все руны можно было получить преобразованием []rune("Hello, Мир").

Функция append играет большую роль в понимании механизма работы слайсов. При вызове append, эта функция сначала проверяет достаточна ли вместительность слайса для добавления в него нового элемента. Если да, то длина слайса увеличивается и добавляется новый элемент. Переданный в функцию слайс и возвращаемый функцией слайс указывают на **один и тот же** базовый массив. Если вместительности слайса не хватает, то **создаётся новый массив** достаточного размера, в него копируется переданный слайс и затем добавляется новый элемент. Теперь возвращаемый функцией слайс конечно же указывает на другой массив, чем тот, на который указывал слайс, переданный в функцию. Новый массив создаётся немного большего размера(обычно увеличивается в два раза), чем это минимально необходимо, чтобы не создавать массивы слишком часто. Это можно увидеть на следующем примере:

In [7]:
var x []int
for i := 1; i < 11; i++ {
    x = append(x, i)
    fmt.Printf("%d cap=%d %v\n", i, cap(x), x)
}

1 cap=1 [1]
2 cap=2 [1 2]
3 cap=4 [1 2 3]
4 cap=4 [1 2 3 4]
5 cap=8 [1 2 3 4 5]
6 cap=8 [1 2 3 4 5 6]
7 cap=8 [1 2 3 4 5 6 7]
8 cap=8 [1 2 3 4 5 6 7 8]
9 cap=16 [1 2 3 4 5 6 7 8 9]
10 cap=16 [1 2 3 4 5 6 7 8 9 10]


append может также использовать и более сложную стратегию роста, чем увеличение размера в 2 раза. Обычно мы не можем знать заранее будет ли в результате вызова append создан новый базовый массив или нет. Следовательно мы не можем предполагать, что после вызова append, изменения внесённые в старый слайс как-то отразятся на новом. Поэтому результат вызова append присваивают переданному ей слайсу, как это было в случае с рунами в начале этого параграфа:  
**runes = append(runes, r)**

append также позволяет добавлять сразу несколько элементов в слайс:

In [8]:
var x []int
x = append(x, 1, 2)
y := []int{3, 4, 5}
x = append(x, y...) // Добавление всего слайса
fmt.Sprint(x)

[1 2 3 4 5]
